<a href="https://colab.research.google.com/github/vcautela/Data_Wrangling/blob/main/Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Setup

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

from sklearn import datasets
from sklearn.preprocessing import scale, minmax_scale, power_transform

from IPython.core.display import HTML
from ipywidgets import interact, widgets

In [2]:
sns.set_theme(
    context='talk',
    style='ticks',
    font_scale=.8,
    rc={
        'figure.figsize': (12,8)
    }
)

In [3]:
#função pra deixar as tabelas na mesma linha (lado a lado)
def display_side_by_side(dataframes:list,titles:list):
    html_str=''
    for df,title in zip(dataframes, titles):
        html_str+=f'<span>{title}:</span>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+="&nbsp"*10
    display(HTML(html_str))

#### Estruturação dos dados

## Merge/Concat/Join

Frequentemente precisamos juntar bases diferentes em um só, para tanto para facilitar análise exploratória quanto para preparar o dataset para modelagem.

As funções `merge`, `concat` e `join` do Pandas nos ajudam nesta parte de estruturação dos dados, e há um tutorial excelente do próprio Pandas [neste link](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare).

### Merge

Este método do pandas é o mais similar aos JOINs do SQL.  
Tabela de exemplo comparativo:


| Merge method   | SQL Join Name    | Description                                         |
|:---------------|:-----------------|:----------------------------------------------------|
| left           | LEFT OUTER JOIN  | Use keys from left frame only                       |
| right          | RIGHT OUTER JOIN | Use keys from right frame only                      |
| outer          | FULL OUTER JOIN  | Use union of keys from both frames                  |
| inner          | INNER JOIN       | Use intersection of keys from both frames           |
| cross          | CROSS JOIN       | Create the cartesian product of rows of both frames |

In [4]:
df1 = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }, index=range(4)
)

df2 = pd.DataFrame(
    {
        "key": ["K1", "K2", "K3", "K4"],
        "C": ["C1", "C2", "C3", "C4"],
        "D": ["D1", "D2", "D3", "D4"],
    }, index=range(1,5)
)

df3 = pd.DataFrame(
    {
        "key": ["K1", "K2", "K3", "K4"],
        "E": ["E1", "E2", "E3", "E4"],
        "F": ["F1", "F2", "F3", "F4"],
    }, index=range(1,5)
)

In [5]:
display_side_by_side([df1,df2, df3], ['df1','df2', 'df3'])

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3
,key,C,D
1,K1,C1,D1
2,K2,C2,D2
3,K3,C3,D3
4,K4,C4,D4
,key,E,F


In [6]:
#merge

df1.merge(df2, on='key', how='inner')

,key,A,B,C,D
0,K1,A1,B1,C1,D1
1,K2,A2,B2,C2,D2
2,K3,A3,B3,C3,D3


In [7]:
# outras formas

df_inner = df1.merge(df2, on='key', how='inner')
df_left = df1.merge(df2, on='key', how='left')
df_right = df1.merge(df2, on='key', how='right')

display_side_by_side(
    [df1, df2, df_inner, df_left, df_right],
    ['df1', 'df2','inner', 'left join', 'right join']
)

In [8]:
#função merge da aula
@interact(method=['inner','left', 'right','outer'])
def merge(method):
  df_merge=df1.merge(df2, how=method)
  display_side_by_side(
      [df1,df2,df_merge],
      ['df1','df2',f'{method}_join']
  )

interactive(children=(Dropdown(description='method', options=('inner', 'left', 'right', 'outer'), value='inner…

### Concat

In [9]:
df_A = df1.iloc[:2]
df_B = df1.iloc[-2:]

display_side_by_side([df_A, df_B],['dfA', 'dfB'])

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
,key,A,B
2,K2,A2,B2
3,K3,A3,B3


In [10]:
# concatenando verticalmente

pd.concat([df_A, df_B])

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [11]:
# concatenando horizontalmente

pd.concat([df_A, df_B], axis=1)

,key,A,B,key,A,B
0,K0,A0,B0,NaN,NaN,NaN
1,K1,A1,B1,NaN,NaN,NaN
2,NaN,NaN,NaN,K2,A2,B2
3,NaN,NaN,NaN,K3,A3,B3


Os valores nulos na concatenação horizontal se deve em função do Index diferente.

Para evitar isso, é adicionado o .reset_index(drop=True) em df_B

In [12]:
# concatenando horizontalmente

pd.concat([df_A, df_B.reset_index(drop=True)], axis=1)

,key,A,B,key,A,B
0,K0,A0,B0,K2,A2,B2
1,K1,A1,B1,K3,A3,B3


### Join

In [13]:
df_left = pd.DataFrame(
    {'A': ['A0', 'A1', 'A2'],
     'B': ['B0', 'B1', 'B2']
    }, index=['K0', 'K1', 'K2']
)
df_right = pd.DataFrame(
    {'C': ['C0', 'C1', 'C2'],
     'D': ['D0', 'D2', 'D3']
    }, index=['K0', 'K2', 'K3']
)

df_join = df_left.join(df_right)

display_side_by_side([df_left, df_right, df_join], ['df_left', 'df_right', 'df_join'])

O join tem por padrão o Left-Join, buscando juntar através do Inner temos:

In [14]:
df_join = df_left.join(df_right, how='inner')

display_side_by_side([df_left, df_right, df_join], ['df_left', 'df_right', 'df_join(inner)'])

A lógica do `groupby` é bem simples e conhecida como **split-apply-combine:**

1. A primeira etapa é separar a tabela (split) em tabelas menores, cada uma com categorias únicas (representando um grupo) da(s) coluna(s) que desejamos agrupar.
2. Aplicar uma função separadamente em cada uma destas tabelas. Pode ser uma função de **agregação** gerando uma métrica de resumo, uma função de **transformação**, modificando os valores, ou uma função de **filtro**, onde filtraremos grupos que se encaixam dentro de uma regra.
3. Por fim, combina-se novamente estas tabelas para formar uma única tabela de saída.

Segue um esquema de exemplo com um função de agregação (soma):


---


![](https://miro.medium.com/max/2400/1*w2oGdXv5btEMxAkAsz8fbg.png)  

In [17]:
# carregando através do seaborn o dataset de gorgetas em um restaurante.

df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [19]:
# agrupando por uma coluna e aplicando um função em todas as colunas

df.groupby('sex').mean()

<ipython-input-19-bdc84b8a8e3f>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('sex').mean()


,total_bill,tip,size
sex,,,
Male,20.744076,3.089618,2.630573
Female,18.056897,2.833448,2.459770


In [20]:
# agrupando por uma coluna e aplicando função em uma coluna-alvo

df.groupby('sex')['tip'].mean()

sex
Male      3.089618
Female    2.833448
Name: tip, dtype: float64

In [21]:
# perceba que acima, ao aplicar a função em coluna alvo, é retornado um pandas.series. Para retornarmos um dataframe podemos aplicar a função em uma lista

df.groupby('sex')[['tip']].mean()

,tip
sex,
Male,3.089618
Female,2.833448


In [22]:
# agrupar por multiplas colunas

df.groupby(['time', 'day'])[['total_bill', 'tip']].mean()

total_bill       tip
time   day                       
Lunch  Thur   17.664754  2.767705
       Fri    12.845714  2.382857
       Sat          NaN       NaN
       Sun          NaN       NaN
Dinner Thur   18.780000  3.000000
       Fri    19.663333  2.940000
       Sat    20.441379  2.993103
       Sun    21.410000  3.255132

In [25]:
# É possível agregar múltiplas funções de agregação suportadas pelo Pandas.

df.groupby('smoker')[['tip']].agg(['min', 'max', 'mean', 'std'])


tip                          
        min   max      mean       std
smoker                               
Yes     1.0  10.0  3.008710  1.401468
No      1.0   9.0  2.991854  1.377190

In [26]:
# é possível agregar funções personalziadas, através de lambda ou função def

df.groupby('smoker')[['tip']].agg(['min', 'max', 'mean', 'std', lambda x: x.mean() + 10])

tip                                     
        min   max      mean       std <lambda_0>
smoker                                          
Yes     1.0  10.0  3.008710  1.401468  13.008710
No      1.0   9.0  2.991854  1.377190  12.991854

In [28]:
def mean_diff(subdf):
  return np.mean(subdf['tip'] / subdf['total_bill'])


df.groupby('sex').apply(mean_diff)

sex
Male      0.157651
Female    0.166491
dtype: float64